In [6]:
import numpy as np


np.random.seed(454)

protein = "1_PARCLIP_AGO1234_hg19"

with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_RNAfold.tab.gz"% protein), "rt") as f:
    fold_data = np.loadtxt(f, skiprows=1) 


    fold = np.zeros((5000,101,fold_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,fold_data.shape[1],101):
        a = fold_data[:,n:(n+101)]
        fold[:,:,(n/101)] = a
        



In [8]:
fold.shape

(5000, 101, 1)

In [12]:
import numpy as np
from Bio import SeqIO
import gzip
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd

np.random.seed(454)


def load_data(path):
    data = gzip.open(os.path.join(path,"sequences.fa.gz"),"rt")
    return data


def get_seq(protein, t_data):
    
    training_data = load_data("data/clip/%s/5000/training_sample_0"% protein)
    x_train = np.zeros((5000,101,4))
    r = 0    

    for record in SeqIO.parse(training_data,"fasta"):
        sequence = list(record.seq)                
        nucleotide = {'A' : 0, 'T' : 1, 'G' : 2, 'C' : 3, 'N' : 4} 
        num_seq = list() #sekvenca v številskem formatu


        for i in range(0,len(sequence)):
                num_seq.append(nucleotide[sequence[i]])


        X = np.zeros((1,len(num_seq),4))

        for i in range (len(num_seq)):
                if num_seq[i] <= 3:
                    X[:,i,num_seq[i]] = 1               

        x_train[r,:,:] = X
        r = r + 1
    
    return x_train


def get_class(protein, t_data):
    y_train = []
    

    if t_data == 'train':
        data = load_data("data/clip/%s/5000/training_sample_0"% protein)

    elif t_data == 'test':
        data = load_data("data/clip/%s/5000/test_sample_0"% protein)


    for record in SeqIO.parse(data,"fasta"):
        v = int((record.description).split(":")[1])
        y_train.append([int(v == 0), int(v != 0)])

    y_train = np.array(y_train)
    return y_train


def get_cobinding(protein, t_data):
    with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_Cobinding.tab.gz"% protein), "rt") as f:
        cobinding_data = np.loadtxt(f, skiprows=1) 
    
    
    cobinding = np.zeros((5000,101,cobinding_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,cobinding_data.shape[1],101):
        a = cobinding_data[:,n:(n+101)]
        cobinding[:,:,(n/101)] = a
    
    return cobinding
    

def get_region (protein, t_data):

    with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_RegionType.tab.gz"% protein), "rt") as f:
        region_data = np.loadtxt(f, skiprows=1) 


    region = np.zeros((5000,101,region_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,region_data.shape[1],101):
        a = region_data[:,n:(n+101)]
        region[:,:,(n/101)] = a
        
    return region


def get_fold (protein, t_data):
   
    with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_RNAfold.tab.gz"% protein), "rt") as f:
        fold_data = np.loadtxt(f, skiprows=1) 
    
    fold = np.zeros((5000,101,fold_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,fold_data.shape[1],101):
        a = fold_data[:,n:(n+101)]
        fold[:,:,(n/101)] = a
    
    
    return fold

In [20]:
score_list = []
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]

for x in range (0,31):

    protein = protein_list[x]
    
    X = np.dstack((get_cobinding(protein,"train"), get_seq(protein,"train"), get_region(protein,"train"), get_fold(protein, "train")))
    y = get_class(protein,"train")
    
    size = X.shape[2]
    
    kf = KFold(n_splits=10) #10 različnih delitev podatkov  
    
    score = []
    for train_index, test_index in kf.split(X):
        
        # sestavljanje modela
        model = Sequential()
        model.add(Conv1D(120, 24,data_format='channels_last', input_shape=(101, size) , strides = 1, padding='valid'))
        model.add(MaxPooling1D(pool_size=24, strides=6, padding='valid'))
        model.add(Conv1D(120, 4, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

          
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
   
    

        y_scores =(model.predict(X_test))# pridobim napovedi za sekvence iz cobinding
        score.append(roc_auc_score(y_test, y_scores))
        
    
    
    print ("%s finishhed" % (protein))
    score_list.append(np.mean(score))
    
    
print(score_list)

1_PARCLIP_AGO1234_hg19 finishhed


KeyboardInterrupt: 

In [8]:
import pandas as pd
import numpy as np


protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
ionmf = [0.927, 0.724, 0.928, 0.932, 0.731, 0.958, 0.964, 0.948, 0.777, 0.973, 0.969, 0.894, 0.829, 0.954, 0.944, 0.952, 0.975, 0.822, 0.790, 0.801, 0.959, 0.869, 0.956, 0.868, 0.913, 0.900, 0.858, 0.949, 0.931, 0.933, 0.904]
score = [0.9229308461970577, 0.7373859595303559, 0.9185030153657214, 0.9216517570803745, 0.7002868366836459, 0.9328408367848035, 0.968480765443396, 0.9610704233808134, 0.7133883127703811, 0.930291845973384, 0.9297965698375279, 0.9300525415845116, 0.908614293424964, 0.980608867261066, 0.9451526335195901, 0.933543787357689, 0.9628050181895601, 0.7837749438422019, 0.6656009182246273, 0.7618671987601008, 0.9684676359870534, 0.8425083385398459, 0.9778942059708827, 0.9490315944769474, 0.8795407520897189, 0.9740336879062095, 0.8697733106206457, 0.9337510003903715, 0.9325289769216644, 0.9310742382453737, 0.9362160341422259]

difference = np.subtract(score,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score, 'iONMF':ionmf, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)


                                                    Difference     Score  \
1_PARCLIP_AGO1234_hg19                               -0.004069  0.922931   
2_PARCLIP_AGO2MNASE_hg19                              0.013386  0.737386   
3_HITSCLIP_Ago2_binding_clusters                     -0.009497  0.918503   
4_HITSCLIP_Ago2_binding_clusters_2                   -0.010348  0.921652   
5_CLIPSEQ_AGO2_hg19                                  -0.030713  0.700287   
6_CLIP-seq-eIF4AIII_1                                -0.025159  0.932841   
7_CLIP-seq-eIF4AIII_2                                 0.004481  0.968481   
8_PARCLIP_ELAVL1_hg19                                 0.013070  0.961070   
9_PARCLIP_ELAVL1MNASE_hg19                           -0.063612  0.713388   
10_PARCLIP_ELAVL1A_hg19                              -0.042708  0.930292   
10_PARCLIP_ELAVL1A_hg19                              -0.039203  0.929797   
12_PARCLIP_EWSR1_hg19                                 0.036053  0.930053   
13_PARCLIP_F

In [ ]:
for x in range (0,31):

    protein = protein_list[x]

    X = np.dstack((get_cobinding(protein,"train"), get_seq(protein,"train"), get_region(protein,"train"), get_fold(protein, "train")))
    y = get_class(protein,"train")

    size = X.shape[2]

    kf = KFold(n_splits=10) #10 različnih delitev podatkov

    score = []
    for train_index, test_index in kf.split(X):

        # sestavljanje modela
        model = Sequential()
        model.add(Conv1D(120,4 ,data_format='channels_last', input_shape=(101, size) , strides = 1, padding='valid'))
        model.add(MaxPooling1D(pool_size=24, strides=2, padding='valid'))
        model.add(Conv1D(120, 4, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)



        y_scores =(model.predict(X_test))# pridobim napovedi za sekvence iz cobinding
        score.append(roc_auc_score(y_test, y_scores))



    print ("%s finishhed" % (protein))


In [1]:
import pandas as pd
import numpy as np

protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
ionmf = [0.927, 0.724, 0.928, 0.932, 0.731, 0.958, 0.964, 0.948, 0.777, 0.973, 0.969, 0.894, 0.829, 0.954, 0.944, 0.952, 0.975, 0.822, 0.790, 0.801, 0.959, 0.869, 0.956, 0.868, 0.913, 0.900, 0.858, 0.949, 0.931, 0.933, 0.904]

score = [0.927228146855331, 0.769494283541073, 0.9313552127427794, 0.9296050749854965, 0.7292741564727823, 0.9369963374903605, 0.9660701142482843, 0.9634841972960959, 0.7557140532300421, 0.933612511482227, 0.9304135125615302, 0.9302392163625471, 0.8967602361845829, 0.9776346381307371, 0.946658674319971, 0.9259490941528631, 0.9491907567623634, 0.8067653522243997, 0.7677425192750433, 0.8047021405346483, 0.9733297070917741, 0.8613936257163806, 0.9767794248136615, 0.9462828423885125, 0.8920009103049378, 0.960460082563338, 0.875195852050852, 0.9315173833880024, 0.9299525323391871, 0.9070087940631355, 0.9276696990129027]


difference = np.subtract(score,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score, 'iONMF':ionmf, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)

                                                    Difference     Score  \
1_PARCLIP_AGO1234_hg19                                0.000228  0.927228   
2_PARCLIP_AGO2MNASE_hg19                              0.045494  0.769494   
3_HITSCLIP_Ago2_binding_clusters                      0.003355  0.931355   
4_HITSCLIP_Ago2_binding_clusters_2                   -0.002395  0.929605   
5_CLIPSEQ_AGO2_hg19                                  -0.001726  0.729274   
6_CLIP-seq-eIF4AIII_1                                -0.021004  0.936996   
7_CLIP-seq-eIF4AIII_2                                 0.002070  0.966070   
8_PARCLIP_ELAVL1_hg19                                 0.015484  0.963484   
9_PARCLIP_ELAVL1MNASE_hg19                           -0.021286  0.755714   
10_PARCLIP_ELAVL1A_hg19                              -0.039387  0.933613   
10_PARCLIP_ELAVL1A_hg19                              -0.038586  0.930414   
12_PARCLIP_EWSR1_hg19                                 0.036239  0.930239   
13_PARCLIP_F